In [1]:
import requests
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import random
import os
import datetime


In [2]:
stock ={"陽明":"2609","長榮":"2603","萬海":"2615","長榮航":"2618","遠雄港":"5607","華航":"2610","台驊投控":"2636","裕民":"2606",
        "益航":"2601" ,"東森":"2614","新興":"2605" , "四維航":"5608" , "台航":"2617" }
month_index = datetime.datetime.strftime(datetime.datetime.today() ,'%Y%m') #index使用

In [3]:
url_eps = [f"https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID={i}&CHT_CAT=MONTH" for i in stock.values()]


In [4]:

headers ={"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}


In [5]:
def get_request_eps(url , headers):
    #request
    r = requests.get(url ,headers=headers)
    r.encoding="utf-8"
    res = r.text
    soup=BeautifulSoup( res ,"lxml")
    data = soup.select_one("#divDetail")
    return data

In [6]:
#EPS 
#需要的最前的時間
def get_eps(data):
    #bs4 解析
    dfs=pd.read_html(data.prettify())[0]#prettify() : 函數將我們的data物件美化所用
    dfs.columns = dfs.iloc[0,:]
    #去除不需要的文字欄位
    indexList =[]
    for i , j in zip(dfs.iloc[:,4] ,dfs.iloc[:,4].index.values):
        if "EPS" in i :
            indexList.append(j)
    df_new=dfs.drop(index=indexList).reset_index().drop(columns="index")
    #找到時間的起點，依照該表單月份的格式去做到搜尋
    eps=df_new.head(1).iloc[:,4].map(lambda x : float(x)).values
    
    return eps  #回傳一個整理好的data

In [11]:
def per_if(x):
    if x<=0 or x==float("inf") or x==float("-inf") :
        return 0
    else:
        return x

In [12]:
df_plus=[]
for name , code  , url in zip(stock ,stock.values() , url_eps ):
    print(url)
    data_eps = get_request_eps(url ,headers )
    time.sleep(15)
    while data_eps is None:
        data_eps = get_request_eps(url ,headers )
        time.sleep(15)
    eps=get_eps(data_eps)
    #price的部分需要完成，才可進行eps\PER的部分
    median_price=pd.read_csv(f"./Data_price/{name}_{code}.csv",index_col=["Unnamed: 0"]).tail(1).iloc[:,2].values
    PER = per_if(float(median_price/eps))
    df=pd.DataFrame({"EPS":eps ,"PER_medain_price":PER} ,index=[month_index]).round(2)
    df_plus.append(df)
    
    
    

https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2609&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2603&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2615&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2618&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=5607&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2610&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2636&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2606&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2601&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2614&CHT_CAT=MONTH
https://goodinfo.tw/StockInfo/ShowK_ChartFlow.asp?RPT_CAT=PER&STOCK_ID=2605&CHT_CAT=MONTH
https://go

In [15]:
#將新的資訊加入原有的資料中
for new ,  key ,name in zip(df_plus  , stock.keys() ,stock.values()) :
    #1.新比對月份是否有重複的
    old=pd.read_csv(f"./Data_EPS和PER/{key}_{name}.csv",index_col=["Unnamed: 0"])#讀入舊的dataFRAME
    new_index=int(new.index.values)
    old_index=old.tail(1).index.values
    if new_index == old_index: 
        """檢查最後ㄧ列的月份是否相同，如相同drop掉，用新的資料"""
        old.drop(index =old_index ,inplace=True )
        #2.將新的月份資料結合
        new_= pd.concat([old ,new])
        
    else:
         #2.將新的月份資料結合
        new_= pd.concat([old ,new])
    
    new_.to_csv(f"./Data_EPS和PER/{key}_{name}.csv")